In [1]:
import os
import openai
from dotenv import load_dotenv
import time
import random
from pymongo import MongoClient
from openai import OpenAI

%load_ext autoreload
%autoreload 2

from db import *
from googlefuncs import *
from oai import *
from workouts import *
from helpers import *
from prompts import *
from function_tools import *
from onboarding import *
from client_registry import ClientRegistry

load_dotenv()

TWILIO_ACC = os.getenv('TWILIO_SID')
TWILIO_TOKEN = os.getenv('TWILIO_TOKEN')
# twilio_client = Client(TWILIO_ACC, TWILIO_TOKEN)
oai_client = instructor.patch(OpenAI(api_key=os.getenv('OPENAI_KEY')))
mongo_client = MongoClient(os.getenv('DB_HOST'))
db = mongo_client['client-management']
# pinecone = pinecone.Pinecone(api_key=os.getenv('PINECONE_KEY'))
# initialize services in main function to be passed around when needed
SERVICE_ACCOUNT_FILE = '../service-assistant-410820-3f5c79fa58a4.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
sheets_service, drive_service = initialize_services(SERVICE_ACCOUNT_FILE, SCOPES)

# ClientRegistry.register_client('twilio_client', twilio_client)
ClientRegistry.register_client('oai_client', oai_client)
ClientRegistry.register_client('mongo_db', db)
ClientRegistry.register_client('sheets_service', sheets_service)
ClientRegistry.register_client('drive_service', drive_service)
# ClientRegistry.register_client('pinecone_client', pinecone)

ModuleNotFoundError: No module named 'dotenv'

In [2]:
common_sense_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'common-sense',
            'description': 'Given a user message, determine unknown entities or items from the message.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'components': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'unknown_component': {'type': 'string',
                                                      'description': 'any unknown component in the message'},
                            },
                        }
                    },
                },
                'required': ['components'],
            },
        }
    }
]

In [91]:
class FunctionEnum(Enum):
    create_new_workout_plan = 'create_new_workout_plan'
    change_exercises_in_workout_plan = 'change_exercises_in_workout_plan'
    question_about_program = 'question_about_program'
    client_indicated_they_completed_workouts = 'client_indicated_they_completed_workouts'
    general_conversation_and_questions = 'general_conversation_and_questions'
    question_about_nutrition = 'question_about_nutrition'
    question_about_trainer_background = 'question_about_trainer_background'
    bad_intent_attempts_to_determine_if_AI_is_being_used = 'bad_intent_attempts_to_determine_if_AI_is_being_used'
    unrelated_to_training_nutrition_or_fitness = 'unrelated_to_training_nutrition_or_fitness'
    does_not_need_a_response = 'does_not_need_a_response'

class Task(BaseModel):
    action_to_take: str
    part_of_user_message: str

class Tasks(BaseModel):
    tasks: List[Task] = []

In [92]:
user_message = 'Congrats on the hyper throw PR!!'

common_sense_1 = {
    'role': 'system',
    'content': '''You are a personal trainer and expert in fitness, nutrition, sports, and weight training. Given a user message: {} follow the steps to determine how to answer the question. Start with Step 1.
    Step 1) Break down the message into tasks
    '''.format(user_message)
}

In [93]:
cs1 = oai_client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=[common_sense_1],
    response_model=Tasks,
    # tools=common_sense_tool,
    # tool_choice={"type": "function", "function": {"name": 'common-sense'}},
    temperature=0,
)

In [94]:
cs1

Tasks(tasks=[Task(action_to_take='acknowledge achievement', part_of_user_message='Congrats on the hyper throw PR!!')])

In [81]:
# break down into sub tasks before asking for unknown components

In [96]:
common_sense_2 = {
    'role': 'system',
    'content': '''You are an expert in fitness, nutrition, sports, and weight training. Given a user message: {} think step by step to determine how to process the question.
    Step 1) Break down the message into tasks
    {}
    Step 2) determine if the message and task is logically sound
    '''.format(user_message, cs1.tasks)
}

In [99]:
class Reasoning(BaseModel):
    contains_unfamiliar_components: bool
    part_of_user_message: str

class Components(BaseModel):
    classification: List[Reasoning] = []


cs2 = oai_client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=[common_sense_2],
    # tools=common_sense_tool,
    # tool_choice={"type": "function", "function": {"name": 'common-sense'}},
    temperature=0,
    response_model=Components
)

In [100]:
cs2

Components(classification=[Reasoning(contains_unfamiliar_components=False, part_of_user_message='Congrats on the hyper throw PR!!')])

In [83]:
class KnownEntity(BaseModel):
    known_or_not: bool  
    what_is_known: str

class Components(BaseModel):
    classification: List[KnownEntity] = []

In [84]:
cs2 = oai_client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=[common_sense_2],
    # tools=common_sense_tool,
    # tool_choice={"type": "function", "function": {"name": 'common-sense'}},
    temperature=0,
    response_model=Components
)

In [85]:
cs2.classification

[KnownEntity(known_or_not=True, what_is_known='acknowledge the achievement')]

In [86]:
class CommonSense(BaseModel):
    does_the_task_make_sense: bool
    reasoning: str

class CommonSenseCheck(BaseModel):
    classification: List[CommonSense] = []

common_sense_3 = {
    'role': 'system',
    'content': '''You are an expert in fitness, nutrition, sports, and weight training. Given a user message: {} think step by step to determine how to process the question.
Step 1) Break down the message into tasks
{}
Step 2) determine if any of the tasks have unknown components
{}
Step 3) determine if the task makes sense
'''.format(user_message, cs1.tasks, cs2.classification)
}

cs3 = oai_client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=[common_sense_3],
    temperature=0,
    response_model=CommonSenseCheck
)

In [87]:
cs3.classification

[CommonSense(does_the_task_make_sense=True, reasoning="Acknowledging someone's achievement, especially in sports or fitness, is a common and positive response. It encourages and celebrates the individual's progress or success.")]

In [88]:
class ToComplete(BaseModel):
    task_should_be_completed: bool
    why_the_task_should_be_completed: str

class CompleteTask(BaseModel):
    classification: List[ToComplete] = []

In [89]:
common_sense_4 = {
    'role': 'system',
    'content': '''You are an expert in fitness, nutrition, sports, and weight training. Given a user message: {} think step by step to determine how to process the question.
    Step 1) Break down the message into tasks
    {}
    Step 2) determine if any of the tasks have unknown components
    {}
    Step 3) determine if the task makes sense
    {}
    Step 4) determine if the task should be completed
    '''.format(user_message, cs1.tasks, cs2.classification, cs3.classification)
}

cs4 = oai_client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=[common_sense_3],
    # tools=common_sense_tool,
    # tool_choice={"type": "function", "function": {"name": 'common-sense'}},
    temperature=0,
    response_model=CompleteTask
)

In [90]:
cs4

CompleteTask(classification=[ToComplete(task_should_be_completed=True, why_the_task_should_be_completed="The task to acknowledge the achievement is clear and relevant, as it involves recognizing and celebrating the user's accomplishment in achieving a personal record (PR) in the hyper throw.")])

In [ ]:
# if the task shouldn't be completed, then the system should tell the user why it shouldn't be completed
# if the task should be completed, then it needs to be delegated to the router